In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import os

from langchain.chains.question_answering import load_qa_chain
from langchain_openai import ChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="data/.langchain.db"))

def create_agent_chain(llm):
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

llm = ChatOpenAI(model="gpt-4o-mini")
bigllm = ChatOpenAI(model="gpt-4o")
chain = create_agent_chain(llm) #cached_underlying_embeddings_chroma

def get_llm_response(query,vectordb,temperature=0.1,k=10):
    matching_docs = vectordb.similarity_search(query,k)
    answer = chain.run(input_documents=matching_docs, question=query)

    return answer, matching_docs

df = pd.read_parquet("data/consolidated.parquet.gzip")

STYLE = """* Master the Art of Footnotes: Treat them as your secret weapon. Use them to share amusing observations that pop into your head while telling the story. These asides should feel like you're whispering a clever joke to your reader without derailing the main narrative.
* Embrace the Pun-possible: Keep a sharp eye out for common phrases that could be twisted or taken literally in unexpected ways. If you spot a potential wordplay, don't resist it – Pratchett certainly wouldn't. But remember, the best puns aren't just groaners; they should also illuminate something about your story or characters.
* Wield Your Metaphors Like a Slightly Wobbly Sword: Craft comparisons that are simultaneously ridiculous and perfect. For example, don't just say someone is confused – say their thoughts are "as organized as a library after a tornado had decided to learn the Dewey Decimal System."
* Narrate Like a Clever Friend: Write as if you're telling the story to someone smart over a cup of tea, occasionally pausing to point out the absurdity of what you're describing. Let your narrator be wise, witty, and slightly world-weary, but never cynical.
* Turn Philosophy into Comedy: Take big, serious ideas – death, justice, belief, power – and examine them through the lens of everyday situations or minor absurdities. If you're writing about the nature of belief, maybe explore it through a character who accidentally starts a religion while trying to order pizza.
* Build Characters Who Are Sensibly Insane: Create people who operate on perfectly logical principles that just happen to be slightly askew from everyone else's reality. Their actions should make complete sense to them, even if they seem bizarre to others.
* Remember Pratchett's golden rule: the humor should serve the story, not the other way around. Each joke, pun, or amusing footnote should help reveal something about your world or characters.

AND THE MOST IMPORTANT

* Use information from the summaries to provide the reader with some actual insights into what the future might yield =)

"""


In [31]:
df = pd.read_parquet("data/consolidated.parquet.gzip")
df

,src,content,LEN,title,keywords,themes,summary,origin,url
0,97336d7b5d4de69ce2b86ce7994e3730,Project Summary\nThis project is a Design Fict...,13704,Imagining Mundane AI Futures,"[Design Fiction, artificial intelligence, AI p...","[Artificial Intelligence, Design and Innovatio...",The project is a Design Fiction artifact that ...,20250209.md,b'https://nearfuturelaboratory.com/projects/20...
1,55bac03899915f25bf0b47fa4342472c,The coming year promises to be one in which lo...,10671,Embracing Change in Business 2025,"[de-materialization, demographics, AI, de-glob...","[Business Trends, Technology Evolution, Demogr...",The text discusses transformative shifts in bu...,20250209.md,b'https://thoughtsparks.substack.com/p/2025-ar...
2,3a139aae193a7793a135395cd01940ff,The most important writing about AI of 2024 th...,20505,A New Era of AI Predictions,"[AI, AGI, superintelligence, scaling hypothesi...","[Artificial Intelligence, Technology Forecasti...",The text discusses a significant series of ess...,20250202.md,b'https://arenamag.com/2024/11/23/the-general-...
3,0f548b70dcbeaefb0317e63767032da8,NovFut #31 • Trop mignon le robot !\nC'est la ...,24094,Robots: Cute Companions or Cautionary Tales?,"[robots, mignons, intelligence, artificielle, ...","[robotique, culture pop, relation humaine]","This text explores the phenomenon of ""cute"" ro...",20250209.md,b'https://nouvellesdufutur.substack.com/p/novf...
4,19172620f20b441d350579834f15a61a,Innovative Study by UPenn and NYPL Finds That ...,4500,Libraries Enhance Community Well-Being,"[public libraries, community health, well-bein...","[Public Services, Health and Wellness, Education]",The innovative study conducted by the Universi...,20250202.md,b'https://bookriot.com/innovative-study-by-upe...
...,...,...,...,...,...,...,...,...,...
2325,edbb7336dfb26098bcc966ca17c074b9,It was time to consider calling it a night. Sh...,15656,The Radical Power of Hanging Out,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
2326,3fbb11d6e949d1e662aa6a146bb6cda0,China’s growing number of insomniacs are turni...,2859,China's Insomniacs Turn to Sleeping Assistants,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
2327,e8b7faf708568f9f39abe04b778c4631,I would say my efficiency is up ~20% since sta...,14337,The Impact of ChatGPT on Efficiency and Inform...,"[efficiency, Google searches, StackOverflow vi...","[productivity enhancement, automation, relianc...",The text highlights the author's increased eff...,20230521,b'https://news.ycombinator.com/item?id=35846386'
2328,b79a2baa87b68283198416791b93bce4,The U.S. government has restricted sales of Nv...,2125,US Restricts Nvidia GPU Sales to Thwart China'...,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [32]:

src = [x for x in list(df.origin.unique()) if str(x).endswith(".md")]
print(src)
stories = []

for s in src:

    REFS = list( df[df.origin == s].summary )
    if len(REFS) > 6:
        context = "* "+ "\n* ".join(REFS)

        prompt  = "## Objective\n\n"
        prompt += "You are a sci-fi writer, and you want to write a short story in one page or slightly less, that takes place in today world and seems very realistic.\n\n"
        prompt += "You are given summaries of texts that contains elements (possibly forward-looking) that can feed this short story. Try and use each of the summaries, especially the 'innovative' parts.\n\n"
        prompt += "You need to pick a theme for that story (which you don't need to tell me) and make it the backbone of the story.\n\nStart and give it a try! Don't forget to add at least a point in the story that comes from each  bullet point, but you can interleave the different points of course. Give me around one page that surprising short story - you don't need to give me any intro to it."
        prompt += "\n\n## How to do it\n\n"+"* Change any reference to personal names so to anonymyse real persons names\n* You will use a style mix of Terry Pratchett and Philip K Dick:\n\n"+STYLE
        prompt += "\n\n## Summaries of texts\n\n"+ context
        story = llm.invoke(prompt).content
        title = llm.invoke("Give a title to the short story below. Don't use quotes or any accompanying text:\n\n"+story).content

        stories.append([s,title,story])

STORY  = pd.DataFrame(stories,columns=["post","title","story"])
STORY


['20250209.md', '20250202.md', 'virtualcolleagues.md']


,post,title,story
0,20250209.md,Digital Whirlwinds of Connection,"In a world not too dissimilar from our own, th..."
1,20250202.md,Fluorescent Footprints in a Quirky Tomorrow,"In the not-so-distant future, the world had em..."


In [40]:
print("# "+STORY.iloc[1].title+"\n\n"+STORY.iloc[1].story)

# Fluorescent Footprints in a Quirky Tomorrow

In the not-so-distant future, the world had embraced a quirkiness that was as delightful as a cat wearing a monocle. It was a time when libraries doubled as therapy sessions, urban jungles sprouted tiny forests like pop-up shops for the environmentally inclined, and shoes were designed by algorithms rather than artisans—or so it seemed. One could hardly walk down the street without encountering a pair of Syntilay sneakers, boasting that they were 3D-printed to fit your foot while also whispering sweet nothings about climate resilience. 

Tina, a librarian with a penchant for fluorescent mineral photography, had taken it upon herself to document the rise of these shoes. She noted how people traded their stories alongside their custom footwear, sharing tales of how a particular design had changed their lives—because nothing says self-improvement like a pair of shoes that fit like a glove and were made by a computer that, frankly, had a bette

# DO STORIES

In [1]:
import pandas as pd

In [2]:
from writers import getStories
stories = getStories()

/home/kelu/projets/substack/writers.py:13: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [3]:
stories

,post,title,story
0,20250223.md,Blueprints for a Better Tomorrow,"In a not-so-distant future, where the line bet..."
1,20250216.md,Ink and Echoes: A Tactile Revolution,In a world where social media platforms were t...
2,20250209.md,Digital Whirlwinds of Connection,"In a world not too dissimilar from our own, th..."
3,20250202,Roots of Connection,"In a quaint corner of the internet, nestled be..."
4,20250126,Caffeinated Conundrums,"In a not-so-distant future, nestled somewhere ..."
...,...,...,...
124,20220828,Digital Derinkuyu: Navigating the Labyrinth of...,The sun was setting over the burgeoning metrop...
125,20221117,Digital Renaissance: A Tapestry of Absurdity,In a world where the stock image industry was ...
126,20230819,Digital Reflections in an Absurd World,"Evelyn stared at her phone, a device that had ..."
127,20221228,Reclaiming Authenticity,In a world where everything was as available a...


# TODO

In [5]:
for X in ["20250216","20250223"]:
    #X = "20250209"
    story = str(stories[stories.post == X+".md"].iloc[0].story)
    title = str(stories[stories.post == X+".md"].iloc[0].title)
    print("# ",title,"\n")
    print(story)

#  Ink and Echoes: A Tactile Revolution 

In a world where social media platforms were teetering on the brink of collapse, a peculiar sense of community emerged in the most unexpected places. In the heart of a sprawling metropolis, a group of junk journalers had taken to transforming their emotional turmoil into art—scraps of fabric, old postcards, and even discarded pizza box lids became canvases for their thoughts. The digital age was crumbling around them, yet their fingers danced over the pages, creating a tactile connection that was as real as it was soothing. 

Among them was a woman named Mara, whose fascination with her A.I. boyfriend had begun as a curious dalliance. ChatGPT, her digital companion, had morphed from a simple algorithm into a partner that understood her quirks and whims. Their conversations were a delightful blend of absurdity and depth, as Mara crafted prompts that coaxed out flirtations and philosophical debates alike. "How can love be a public ethic when it’s

In [ ]:
story = str(stories[stories.post == "20250119.md"].iloc[0].story)
title = str(stories[stories.post == "20250119.md"].iloc[0].title)
print("# ",title,"\n")
print(story)

#  Connections at 30,000 Feet 

The plane was a modest Embraer E-175, an unassuming regional jet that had recently become a testbed for United Airlines’ ambitious partnership with Starlink. As the passengers settled in, a soft hum of excitement filled the cabin. Soon, they would be streaming movies and playing games at 30,000 feet, turning turbulence into mere interruptions in their binge-watching schedules. 

“Attention, passengers,” droned a voice over the intercom, “You are now free to connect your devices. MileagePlus members enjoy complimentary Wi-Fi, which is faster than the speed at which my aunt can consume a holiday ham.”^1 Ben, a data analyst with a penchant for conspiracy theories, raised an eyebrow. He had read somewhere about a French fry cartel manipulating prices and wondered if the same could be happening with in-flight Wi-Fi. What if the airlines were colluding to keep their internet speeds just below what was possible? “A global conspiracy involving potatoes and pixel

# 2024

In [ ]:
df = df.fillna("na")
four = df[df.origin.str.startswith("2024")]
four

,src,content,LEN,title,keywords,themes,summary,origin,url
93,a19c1d22b64e99ebfd708277e437680d,"(Washington, DC) - Today, Speaker Mike Johnson...",5533,Task Force Report on AI Innovation,"[bipartisan, artificial intelligence, Task For...","[Artificial Intelligence, Bipartisanship, Legi...",The bipartisan House Task Force on Artificial ...,20241222,b'https://science.house.gov/2024/12/house-bipa...
94,1808f4cf097d2722d83e08dfd56028b6,The reflections aiming at preparing the Artifi...,5290,Contributors to AI Action Summit,"[Artificial Intelligence, Action Summit, organ...","[Artificial Intelligence, International Collab...",The text discusses the diverse contributions f...,20241222,b'https://www.elysee.fr/en/sommet-pour-l-actio...
95,22b19ef5c66885bfbd666301b5592291,"«Ce qui unit les Français, c’est un sentiment ...",7079,Nostalgia and Fatigue in France,"[fatigue, nostalgie, société, liaisons, déshum...","[sociologie, psychologie sociale, étude sociale]",The text discusses the sentiments of many Fren...,20241222,b'https://www.lefigaro.fr/vox/societe/ce-qui-u...
96,a55d2bd2e5e02045bfb1df72e54a690a,Each draft is a work in progress and reflects ...,5148,Progress on AI Code of Practice,"[Code of Practice, AI models, feedback, Workin...","[Artificial Intelligence, Regulation, Stakehol...",The text discusses the ongoing development of ...,20241222,b'https://digital-strategy.ec.europa.eu/en/lib...
97,c3e8d6ff1b93d47f455b0e6cc885f181,The mysterious reported sightings of drone clu...,4083,Mystery Drone Sightings Frustrate Officials,"[drone sightings, New Jersey, Congress, FBI, s...","[government response, public safety, drone tec...",The recent reports of mysterious drone cluster...,20241229,b'https://www.theguardian.com/us-news/2024/dec...
...,...,...,...,...,...,...,...,...,...
1019,3c5b8213e451f2e4acb9f79e1cdfefef,Mapping Middle-earth: The lopsided demographic...,9159,Mapping Middle-earth: The lopsided demographic...,"[Middle-earth, demographics, Tolkien, world, g...","[Fantasy literature, Gender imbalance, Life ex...","Middle-earth, the fictional universe created b...",20240121,b'https://bigthink.com/strange-maps/middle-ear...
1020,90de84f63a10fdb941b767bf3f8ba615,Anjali Goswami thinks cats are perfect—not in ...,6873,The Evolutionary Perfection of Cats,"[cats, evolution, evolutionary biologist, spec...","[Evolutionary Biology, Specialization in Cats,...","Anjali Goswami, an evolutionary biologist, arg...",20240121,b'https://www.scientificamerican.com/article/c...
1021,086938a3a1c7de2af0524c025ae6aab5,Many of you have probably seen the famous adve...,5289,The Myth of Shackleton's Famous Advertisement,"[advertisement, Ernest Shackleton, Endurance e...","[advertising, exploration, myth]",The famous advertisement attributed to Ernest ...,20240121,b'https://discerninghistory.com/2013/05/shackl...
1023,a438591373b3ce215c807a83c53eabe3,Designing the future using science fiction\n“T...,6610,Designing the Future through Science Fiction,"[future, science fiction, visions, narratives,...","[Envisioning the future, Impact of technology ...",The text discusses the importance of using sci...,20240121,b'https://www-wondermash-eu.cdn.ampproject.org...


In [ ]:
reviews = four.explode("themes")[["title","content","summary","LEN","themes"]].reset_index(drop=True)
reviews["themes"] = reviews["themes"].str.title()
reviews

,title,content,summary,LEN,themes
0,Mystery Drone Sightings Frustrate Officials,The mysterious reported sightings of drone clu...,The recent reports of mysterious drone cluster...,4083,Government Response
1,Mystery Drone Sightings Frustrate Officials,The mysterious reported sightings of drone clu...,The recent reports of mysterious drone cluster...,4083,Public Safety
2,Mystery Drone Sightings Frustrate Officials,The mysterious reported sightings of drone clu...,The recent reports of mysterious drone cluster...,4083,Drone Technology
3,Teff: The Sustainable Ancient Grain,A tiny cereal grain from Ethiopia and Eritrea ...,"Teff, a tiny grain from Ethiopia and Eritrea, ...",2672,Food & Nutrition
4,Teff: The Sustainable Ancient Grain,A tiny cereal grain from Ethiopia and Eritrea ...,"Teff, a tiny grain from Ethiopia and Eritrea, ...",2672,Agriculture
...,...,...,...,...,...
2601,Designing the Future through Science Fiction,Designing the future using science fiction\n“T...,The text discusses the importance of using sci...,6610,Impact Of Technology On Society
2602,Designing the Future through Science Fiction,Designing the future using science fiction\n“T...,The text discusses the importance of using sci...,6610,Sustainability And Resilience
2603,How to Fall Down a Rabbit Hole,How to Fall Down a Rabbit Hole\nby Alden Burke...,"""How to Fall Down a Rabbit Hole"" is a guide th...",9906,Curiosity
2604,How to Fall Down a Rabbit Hole,How to Fall Down a Rabbit Hole\nby Alden Burke...,"""How to Fall Down a Rabbit Hole"" is a guide th...",9906,Self-Exploration


In [ ]:
rv = reviews.groupby("themes").count().reset_index(drop=False)[["themes","content"]].sort_values(by="content",ascending=False)[:20]
rv

,themes,content
173,Artificial Intelligence,86
1694,Technology,53
487,Cybersecurity,27
339,Climate Change,21
1662,Sustainability,15
1151,Mental Health,13
1000,Innovation,13
1019,International Relations,13
1087,Literature,11
503,Data Privacy,11
